In [1]:
%%capture
# !pip install --upgrade pip
# !pip install --upgrade tensorflow
!pip install transformers[sentencepiece]
!pip install datasets
# !pip install nlp
!pip install --upgrade jupyter
!pip install ipywidgets

In [2]:
import gc
import pandas as pd
import numpy as np


from tqdm import tqdm
from datasets import load_dataset

# from transformers import TFAutoModelForSequenceClassification
# from transformers import TFAutoModel

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1,L2
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [3]:
print("Tensorflow version " + tf.__version__)

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Tensorflow version 2.14.0
Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2023-12-19 14:41:47.299703: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:41:47.299855: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:41:47.299932: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:41:47.300010: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:41:47.300082: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:41:47.300324: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:41:47.300426: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
from transformers import AutoTokenizer, TFAutoModel
# checkpoint = 'distilbert-base-multilingual-cased'
# checkpoint = 'xlm-roberta-base'
# checkpoint='bert-base-multilingual-cased'
# checkpoint = 'microsoft/mdeberta-v3-base'
checkpoint = 'symanto/xlm-roberta-base-snli-mnli-anli-xnli'
# checkpoint='jplu/tf-xlm-roberta-large'
# checkpoint="vicgalle/xlm-roberta-large-xnli-anli"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=3,
    restore_best_weights=True)

lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=400,
    decay_rate=0.9,
    staircase=True)

lr_reduce=ReduceLROnPlateau(monitor='val_loss',
                           factor=0.4,
                           min_delta=0.01,
                           patience=5,
                           min_lr=1e-8,
                           verbose=1)
# optimizer = Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.98,epsilon=1e-9)
# optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.98,epsilon=1e-9)

In [7]:
optimizer = Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.98,epsilon=1e-9)

tf.keras.backend.clear_session()
with strategy.scope():
    pretrained_model = TFAutoModel.from_pretrained(checkpoint,from_pt=True)
    pretrained_model.trainable=True
    gc.collect()
    def build_model():
        input_1 = layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
        input_2 = layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
#         input_3 = layers.Input(shape=(None,), dtype=tf.int32, name='token_type_ids')

        x = pretrained_model([input_1,input_2]).last_hidden_state[:,0,:]
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(rate=0.3)(x)
        
#         x = layers.Dense(units=108, activation='gelu',kernel_regularizer=L2(0.005))(x)
#         x = layers.BatchNormalization()(x)
#         x = layers.Dropout(rate=0.3)(x)
        
        x = layers.Dense(units=72, activation='gelu',kernel_regularizer=L2(0.005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(rate=0.2)(x)
        
        x = layers.Dense(units=48, activation='gelu',kernel_regularizer=L2(0.005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(rate=0.1)(x)
        
        x = layers.Dense(units=24, activation='gelu',kernel_regularizer=L2(0.005))(x)
        x = layers.BatchNormalization()(x)
#         x = layers.Dropout(rate=0.1)(x)
        
        out = layers.Dense(units=3, activation='softmax')(x)

        model = tf.keras.Model(inputs=[input_1,input_2], outputs=out)
        return model
    model = build_model()
    model.compile(optimizer=optimizer,
                  loss=SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

2023-12-19 14:42:43.870459: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:42:43.870575: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['classifier.out_proj.weight', 'roberta.embeddings.position_ids', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# mnli = load_dataset("glue", "mnli")
# snli = load_dataset("snli")
xnli = load_dataset('xnli', 'all_languages',streaming=False,split='train')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [9]:
def mono_lang_dataset_processing(dataset):
    df = [(record['premise'], record['hypothesis'], record['label'], 'en') for record in tqdm(dataset) if record['label'] in {0, 1, 2}]
    df = pd.DataFrame(df, columns=['premise', 'hypothesis', 'label','lang_abv'])

    return df

def multi_lang_dataset_processing(dataset):
    data = [
    (
        record['premise'].get(lang, None),
        trans,
        record['label'],
        lang
    )
    for record in tqdm(dataset)
    if record['hypothesis'] and record['premise'] and record['label'] in {0, 1, 2}
    for lang, trans in zip(record['hypothesis']['language'], record['hypothesis']['translation'])
    if record['premise'].get(lang, None) is not None
    ]

    df = pd.DataFrame(data, columns=['premise', 'hypothesis', 'label', 'lang_abv'])

    return df

In [10]:
train_set=multi_lang_dataset_processing(xnli)
# train_set=pd.concat([train_set,mono_lang_dataset_processing(mnli['train'])])
# train_set=pd.concat([train_set,mono_lang_dataset_processing(snli['train'])])
# train_set.to_csv('add_data_1.csv',index=True)

100%|██████████| 392702/392702 [01:00<00:00, 6486.58it/s]


In [11]:
df_train=pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
df_test =pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

train_set_1=pd.concat([train_set[:90000],df_train[:10000]],ignore_index=True)
# train_set_1=df_train[:10000]
# train_set_1=train_set_1[train_set_1['premise'].apply(lambda x: len(tokenizer(x)['input_ids']) <= 256)]
# print(len(train_set_1))

In [12]:
# max_length=272
def tokenizer_function(dataset):
    encode=tokenizer.batch_encode_plus(dataset[['premise','hypothesis']].values.tolist(),padding='max_length',truncation=True,max_length=256,return_tensors="np")
    return dict(encode)

np.object = object
x_train = tokenizer_function(train_set_1)
x_valid = tokenizer_function(df_train[10000:])
x_test  = tokenizer_function(df_test)

In [13]:
auto = tf.data.experimental.AUTOTUNE
def build_dataset(x, y, mode, batch_size):
    if mode == "train":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices((x, y))
            # .repeat()
            .shuffle(2048,reshuffle_each_iteration = True)
            .batch(batch_size)
            .prefetch(auto)
        )
    elif mode == "valid":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices((x, y))
            .batch(batch_size)
            .cache()
            .prefetch(auto)
        )
    elif mode == "test":
        dataset = (
            tf.data.Dataset
            .from_tensor_slices(x)
            .batch(batch_size)
        )
    else:
        raise NotImplementedError
    return dataset

In [14]:
BATCH_SIZE=16 * strategy.num_replicas_in_sync
train_dataset = build_dataset(x_train, train_set_1['label'], "train", BATCH_SIZE)
valid_dataset = build_dataset(x_valid, df_train[10000:]['label'], "valid", BATCH_SIZE)
test_dataset  = build_dataset(x_test, None, "test", BATCH_SIZE)
gc.collect()

0

In [15]:
# del xnli
# del train_set
del train_set_1
del df_train
del df_test
del x_train
del x_valid
del x_test
gc.collect()

0

In [16]:
gc.collect()

0

In [17]:
# with strategy.scope():
#     model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_dataset,
          validation_data=valid_dataset,
          callbacks=[early_stopping,lr_reduce],
          epochs=20,
          verbose=True)

2023-12-19 14:47:05.876595: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:05.896828: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:05.916708: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:05.936387: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:05.956999: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:05.978100: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:05.998354: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

Epoch 1/20


/usr/local/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 192001536 elements. This may consume a large amount of memory.
  warnings.warn(


2023-12-19 14:47:40.902884: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
2023-12-19 14:47:43.939787: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:43.943208: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:43.943316: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:43.943466: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:43.943600: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-19 14:47:43.943708: E ./tensorflow/compiler/xla/stream_executor/st

  2/782 [..............................] - ETA: 2:27 - loss: 2.7673 - accuracy: 0.3516    

...

782/782 [==============================] - 305s 275ms/step - loss: 1.6262 - accuracy: 0.3532 - val_loss: 1.1577 - val_accuracy: 0.3472 - lr: 9.0000e-04
Epoch 2/20
782/782 [==============================] - 148s 189ms/step - loss: 1.1248 - accuracy: 0.3629 - val_loss: 1.1230 - val_accuracy: 0.3420 - lr: 7.2900e-04
Epoch 3/20
 11/782 [..............................] - ETA: 2:24 - loss: 1.1087 - accuracy: 0.3679

2023-12-19 14:54:43.261827: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


782/782 [==============================] - 149s 191ms/step - loss: 1.1071 - accuracy: 0.3643 - val_loss: 1.1071 - val_accuracy: 0.3476 - lr: 5.9049e-04
Epoch 4/20
334/782 [===========>..................] - ETA: 1:23 - loss: 1.1039 - accuracy: 0.3590

2023-12-19 14:58:13.608112: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


782/782 [==============================] - 163s 209ms/step - loss: 1.1024 - accuracy: 0.3650 - val_loss: 1.1041 - val_accuracy: 0.3458 - lr: 4.7830e-04


In [18]:
pred=model.predict(test_dataset)
sample_submission=pd.read_csv('/kaggle/input/contradictory-my-dear-watson/sample_submission.csv')
sample_submission['prediction']=np.argmax(pred,axis=1)
sample_submission.to_csv('submission.csv',index=False)

2023-12-19 14:59:58.775202: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


41/41 [==============================] - 19s 196ms/step
